# AWS Lab Automation

By Gion Rubitschung (student16)

# Import Section

In [ ]:
import json
import boto3
import re

# Configuration and setup

In [2]:
PROFILE = "student16"
REGION = "eu-central-1"
STUDENT_ID = "student16"

GENERATE_FILE_LAMBDA = "GenerateStudentFile"
GENERATE_HASH_LAMBDA = "GenerateHash"
S3_BUCKET = "bfh-student-task-bucket"
DYNAMO_TABLE = "bfh-student-results"

In [3]:
session = boto3.Session(profile_name=PROFILE)
lambda_client = session.client("lambda")
s3_client = session.client("s3")
dynamo_client = session.client("dynamodb")

# Generate student file and get the file path

In [4]:
payload = {"student_id": STUDENT_ID}
response = lambda_client.invoke(
    FunctionName=GENERATE_FILE_LAMBDA,
    Payload=json.dumps(payload).encode(),
)
response

{'ResponseMetadata': {'RequestId': 'ebccf925-b92b-4aab-be90-0854ac0779f6',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 28 Apr 2025 11:28:07 GMT',
   'content-type': 'application/json',
   'content-length': '114',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'ebccf925-b92b-4aab-be90-0854ac0779f6',
   'x-amzn-remapped-content-length': '0',
   'x-amz-executed-version': '$LATEST',
   'x-amzn-trace-id': 'Root=1-680f6646-3a9ff42771208e043eba5e82;Parent=5b2b2e130cfc5bd3;Sampled=0;Lineage=1:7f98cead:0'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'ExecutedVersion': '$LATEST',
 'Payload': <botocore.response.StreamingBody at 0x107984b80>}

In [5]:
payload_response = response["Payload"].read().decode("utf-8")
body = json.loads(json.loads(payload_response)["body"])
body

{'message': 'File created', 'file_key': 'incoming/student16/student16.txt'}

In [6]:
file_path = body["file_key"]
file_path

'incoming/student16/student16.txt'

# Download the student file and retrieve the secret number

In [7]:
obj = s3_client.get_object(Bucket=S3_BUCKET, Key=file_path)
obj

{'ResponseMetadata': {'RequestId': '5XFSR6RYG6068TMR',
  'HostId': 'n65wiIDtip0voGWiQwSUWbF5/5QdokZJkDTcVK2wMbMc4TM2r8VN1eco4tyvAB4bSKHqQvA8SCRpWnxecyDs48d8ToiTPLhAkDTYjlr+RFY=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'n65wiIDtip0voGWiQwSUWbF5/5QdokZJkDTcVK2wMbMc4TM2r8VN1eco4tyvAB4bSKHqQvA8SCRpWnxecyDs48d8ToiTPLhAkDTYjlr+RFY=',
   'x-amz-request-id': '5XFSR6RYG6068TMR',
   'date': 'Mon, 28 Apr 2025 11:28:08 GMT',
   'last-modified': 'Mon, 28 Apr 2025 11:28:08 GMT',
   'etag': '"e6eaefbc49d4c4195151afb0a96d1a19"',
   'x-amz-checksum-crc64nvme': 'locZ4nwvgS0=',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'x-amz-server-side-encryption': 'AES256',
   'accept-ranges': 'bytes',
   'content-type': 'text/plain',
   'content-length': '40',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'AcceptRanges': 'bytes',
 'LastModified': datetime.datetime(2025, 4, 28, 11, 28, 8, tzinfo=tzutc()),
 'ContentLength': 40,
 'ETag': '"e6eaefbc49d4c4195151afb0a96d1a19"',
 'ChecksumCRC64NVME':

In [8]:
student_file = obj["Body"].read().decode().strip()
student_file

'Student ID: student16\nYour Number: 9794'

In [9]:
pattern = r"Your Number: (\d+)"
match = re.search(pattern, student_file)
secret_number = match.group(1)
secret_number

'9794'

# Generate the student hash

In [10]:
payload = {"student_id": STUDENT_ID, "number": secret_number}
response = lambda_client.invoke(
    FunctionName=GENERATE_HASH_LAMBDA,
    Payload=json.dumps(payload).encode(),
)
response

{'ResponseMetadata': {'RequestId': '044b1814-2109-47c5-b68e-fbea78f49e35',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Mon, 28 Apr 2025 11:28:09 GMT',
   'content-type': 'application/json',
   'content-length': '83',
   'connection': 'keep-alive',
   'x-amzn-requestid': '044b1814-2109-47c5-b68e-fbea78f49e35',
   'x-amzn-remapped-content-length': '0',
   'x-amz-executed-version': '$LATEST',
   'x-amzn-trace-id': 'Root=1-680f6648-7756e75e48403d1f53da5ec2;Parent=385dcdf130c0a494;Sampled=0;Lineage=1:5e4bfffb:0'},
  'RetryAttempts': 0},
 'StatusCode': 200,
 'ExecutedVersion': '$LATEST',
 'Payload': <botocore.response.StreamingBody at 0x107a582b0>}

In [11]:
payload_response = response["Payload"].read().decode("utf-8")
body = json.loads(json.loads(payload_response)["body"])
body

{'md5_hash': 'd97774cbf0f97388141a5e7b57454fa0'}

In [12]:
md5_hash = body["md5_hash"]
md5_hash

'd97774cbf0f97388141a5e7b57454fa0'

# Create the student file with the obtained hash

In [13]:
key = f"hash/{STUDENT_ID}/{STUDENT_ID}_hash.txt"
s3_client.put_object(Bucket=S3_BUCKET, Key=key, Body=md5_hash.encode())

{'ResponseMetadata': {'RequestId': 'FFAYAB517TNFNPK7',
  'HostId': 'RLAGpKT/qW9F3nOlio/7USzhRMTMlH1WQh6XyVMf9tpyLYnbsdpHO9qJ5EugMMX46YgA46ea0xWKUg4tftfh0Xc5hV4S5FS9SZhXCkotY1M=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'RLAGpKT/qW9F3nOlio/7USzhRMTMlH1WQh6XyVMf9tpyLYnbsdpHO9qJ5EugMMX46YgA46ea0xWKUg4tftfh0Xc5hV4S5FS9SZhXCkotY1M=',
   'x-amz-request-id': 'FFAYAB517TNFNPK7',
   'date': 'Mon, 28 Apr 2025 11:28:10 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"dd0f122ac1d2a3a1ac407f32ff63f4f0"',
   'x-amz-checksum-crc32': 'JFr0yQ==',
   'x-amz-checksum-type': 'FULL_OBJECT',
   'content-length': '0',
   'server': 'AmazonS3'},
  'RetryAttempts': 0},
 'ETag': '"dd0f122ac1d2a3a1ac407f32ff63f4f0"',
 'ChecksumCRC32': 'JFr0yQ==',
 'ChecksumType': 'FULL_OBJECT',
 'ServerSideEncryption': 'AES256'}

# Insert the hash into dynamodb and verify it exists

In [14]:
dynamo_client.put_item(
    TableName=DYNAMO_TABLE,
    Item={
        "student_id": {"S": STUDENT_ID},
        "md5_hash": {"S": md5_hash},
    },
)

{'ResponseMetadata': {'RequestId': 'RBUI177UVE990OMVJGN72TM3MNVV4KQNSO5AEMVJF66Q9ASUAAJG',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'server': 'Server',
   'date': 'Mon, 28 Apr 2025 11:28:09 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive',
   'x-amzn-requestid': 'RBUI177UVE990OMVJGN72TM3MNVV4KQNSO5AEMVJF66Q9ASUAAJG',
   'x-amz-crc32': '2745614147'},
  'RetryAttempts': 0}}

In [15]:
dynamo_client.get_item(
    TableName=DYNAMO_TABLE,
    Key={
        'student_id': {'S': STUDENT_ID}
    }
).get("Item")

{'md5_hash': {'S': 'd97774cbf0f97388141a5e7b57454fa0'},
 'student_id': {'S': 'student16'}}